## 代码说明：
### 1. 实现了resnext和SE module，对基础模块basicblock和bottleneck加入SE
### 2. 并基于现有的ResNet代码结构增加了SE功能，通过同一个对象可以按需创建是否带SE的ResNet或resnext网络结构
### 3. 封装了封装了两大类接口get_resnet()和get_resnext(), 可以通过入参创建是否带SE的各种规格resnet和resnext网络结构

### 导入库

In [1]:
#import myresnet
from myresnet import *

### 加载图片，并预处理

In [2]:
filename = 'dog.jpg'
# 预处理,得到输入数据
input_batch = preprocess(filename)

### 创建ResNet152，并与官方接口对比，验证模型正确性
#### 结论： 与官方结果一致

In [4]:
# 创建并加载模型预训练参数
my_resnet152 = get_resnet(mode = 152, use_se = False, pretrained = False, progress = True)
my_resnet152.load_state_dict(torch.load('C:/Users/Administrator/.cache/torch/checkpoints/'+'resnet152-b121ed2d.pth', map_location='cpu'))
res, res_sftmx = inference(my_resnet152, input_batch)
print("自己实现ResNet152结果:{}".format(res))

model_resnet152 = get_official_model('resnet152')
res_official, res_official_sftmx = inference(model_resnet152, input_batch)
print("官方实现ResNet152结果:{}".format(res_official))

isEqual = torch.equal(res_sftmx, res_official_sftmx)
print("两种实现结果是否一致:{}".format(isEqual))

torch.Size([1, 1000])
argmax_res:258
自己实现ResNet152结果:258
torch.Size([1, 1000])
argmax_res:258
官方实现ResNet152结果:258
两种实现结果是否一致:True


### 创建resnext101_32x8d，并与官方接口对比，验证模型正确性
#### 创建方式：通过自己封装好的接口get_resnext()创建
#### 结论： 与官方结果一致

In [6]:
model = get_official_model('resnext101_32x8d')
res_official, res_official_sftmx = inference(model, input_batch)
print("官方实现resnext101_32x8d结果:{}".format(res_official))

my_resnext101 = get_resnext(mode=1, use_se=False, pretrained=False, progress=True)       
#model_path = 'seresnext50_32x4d-0521-b0ce2520.pth'
model_path = 'resnext101_32x8d-8ba56ff5.pth'
base_path = 'C:/Users/Administrator/.cache/torch/checkpoints/'
state_dict = torch.load(base_path + model_path, map_location='cpu')
my_resnext101.load_state_dict(state_dict, strict=False)
res, res_sftmx = inference(my_resnext101, input_batch)
print("自己实现resnext101_32x8d结果:{}".format(res))

isEqual = torch.equal(res_sftmx, res_official_sftmx)
print("两种实现结果是否一致:{}".format(isEqual))


torch.Size([1, 1000])
argmax_res:258
官方实现resnext101_32x8d结果:258
torch.Size([1, 1000])
argmax_res:258
自己实现resnext101_32x8d结果:258
两种实现结果是否一致:True


### 创建SE-resnext50，并基于开源模型进行对比验证
#### 创建方式：通过自己封装好的get_resnext()接口创建
#### 开源模型repo：https://github.com/osmr/imgclsmob
#### 结论：
##### 1. 模型层数相等，都为384层
##### 2. 其中36层参数不一致，原因为开源代码与自己代码在make_layer环节中基础block的代码实现顺序及结构不一致，主要为SE模块， 降采样模块

In [9]:
my_seresnext50 = get_resnext(mode=0, use_se=True, pretrained=False, progress=True)
my_dict = my_seresnext50.state_dict()
model_path = 'seresnext50_32x4d-0521-b0ce2520.pth'
#model_path = 'resnext101_32x8d-8ba56ff5.pth'
base_path = 'C:/Users/Administrator/.cache/torch/checkpoints/'
state_dict = torch.load(base_path + model_path, map_location='cpu')
count = 0
# 由于模型所有层的名称不一致，需要逐层判断其形状来统计哪些层定义不一致
for my_key, key in zip(list(my_dict.keys()), list(state_dict.keys())):
    if my_dict[my_key].shape != state_dict[key].shape:
        count += 1
        print("第{}个不一致层:".format(count))
        print("自己模型层名称:{},参数形状:{}".format(my_key, my_dict[my_key].shape))
        print("开源模型层名称:{},参数形状:{}".format(key, state_dict[key].shape))
        
    else:
        my_dict[my_key] = copy.deepcopy(state_dict[key])
print("共有{}层参数形状不一致".format(count))
my_seresnext50.load_state_dict(my_dict, strict=False)
#my_resnext.load_state_dict(state_dict, strict=False)
res, res_sftmx = inference(my_seresnext50, input_batch)
print("自己实现resnext101_32x8d结果:{}".format(res))

isEqual = torch.equal(res_sftmx, res_official_sftmx)
print("两种实现结果是否一致:{}".format(isEqual))

第1个不一致层:
自己模型层名称:layer1.0.downsample.0.weight,参数形状:torch.Size([256, 64, 1, 1])
开源模型层名称:features.stage1.unit1.se.conv1.weight,参数形状:torch.Size([16, 256, 1, 1])
第2个不一致层:
自己模型层名称:layer1.0.downsample.1.weight,参数形状:torch.Size([256])
开源模型层名称:features.stage1.unit1.se.conv1.bias,参数形状:torch.Size([16])
第3个不一致层:
自己模型层名称:layer1.0.downsample.1.bias,参数形状:torch.Size([256])
开源模型层名称:features.stage1.unit1.se.conv2.weight,参数形状:torch.Size([256, 16, 1, 1])
第4个不一致层:
自己模型层名称:layer1.0.downsample.1.running_var,参数形状:torch.Size([256])
开源模型层名称:features.stage1.unit1.identity_conv.conv.weight,参数形状:torch.Size([256, 64, 1, 1])
第5个不一致层:
自己模型层名称:layer1.0.downsample.1.num_batches_tracked,参数形状:torch.Size([])
开源模型层名称:features.stage1.unit1.identity_conv.bn.weight,参数形状:torch.Size([256])
第6个不一致层:
自己模型层名称:layer1.0.se_module.fc1.weight,参数形状:torch.Size([16, 256, 1, 1])
开源模型层名称:features.stage1.unit1.identity_conv.bn.bias,参数形状:torch.Size([256])
第7个不一致层:
自己模型层名称:layer1.0.se_module.fc1.bias,参数形状:torch.Size([16])
开源模型层名称:features.sta